# ТЕСТИРОВАНИЕ РЕШЕНИЯ
Полученного в результате выполнения 'main.ipynb'

## Технические ограничения
Рекомендуется запуск на **google colab**, т.к. там настроена вся необходимая среда.
Крайне не рекомендуется запускать на Windows - потребуется установить и настроить кучу зависимостей для dlib, Visual Studio, C++ - Packages CMake tools для Windows, ffmpeg, настроить переменные среды и др.

Для 20 минутного видео необходим объём ОЗУ 24 гигабайта только на тензор видео. Таких ресурсов у меня нет. Обрезал видео до 3 минут и смог запустить пример на google colab

## Author
Domnenko Aleksey 2021
*   telegram: @domnenko_a_n
*   email: domnenko@mail.ru



## Внимание!
Не забудьте указать в настройках свои пути до папок.

================================================================================


# ПОДКЛЮЧЕНИЕ

## Установка необходимых пакетов

In [1]:
!pip install -q sk-video

     |████████████████████████████████| 2.3MB 8.4MB/s 


## Подключение пакетов

In [2]:
import os.path
import time
import pandas as pd
import cv2
import numpy as np
from skimage import io  # Загрузка изображения
import skvideo.io  # Загрузка видео
from scipy.spatial import distance  # Расчёт Евклидового расстояния
from IPython.display import HTML  # Для просмотра видео
import matplotlib.pyplot as plt
import dlib  # Поиск и выделение лиц

## Монтируем Google drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# НАСТРОЙКИ

In [4]:
# --- ВНИМАНИЕ! ---
# Укажите свой путь до папки с файлами
DIR = '/content/drive/MyDrive/Colab Notebooks/ОБУЧЕНИЕ/УИИ/Поиск Джима Керри на видео/files'  # Абсолютный путь до папки с файлами

# КЛАСС ДЛЯ РАБОТЫ С ВИДЕОРОЛИКОМ

### **Метод __init__()**
Настройки и инициализация объектов dlib

**shape_predictor**

Этот объект представляет собой инструмент, который берет область изображения, содержащую некоторый объект, и выводит набор точек, определяющих позу объекта. Классическим примером этого является прогнозирование позы человеческого лица, когда вы берёте изображение человеческого лица в качестве входных данных и, как ожидается, определите расположение важных лицевых ориентиров, таких как уголки рта и глаз, кончик носа и т. Д. вперед.

**face_recognition_model_v1**

Этот объект переводит человеческие лица в 128 разрядные векторы, где изображения одного и того же человека расположены рядом друг с другом, а изображения разных людей нанесены на карту далеко друг от друга. Конструктор загружает модель распознавания лиц из файла

**get_frontal_face_detector**

Возвращает детектор лиц по умолчанию. Координаты лица, обнаруженного на изображении

### **Метод getRectangle()**

Получаем тензор, содержащий только область лица.
Возвращает тензор изображения и дескриптор.

### **Метод euclideanDistance()**
Рассчитываем Евклидово расстояние между векторами и возвращаем меру схожести.

### **Метод saveMovie()**
Получает сохраняет виеоролик с кадрами из списка self.frame_list

### **Метод showImage()**
Отображает 3 изображения из входного списка


In [5]:
class DanFace:
  def __init__(self):
    self.frame_list = []  # Список кадров, на которых был найден Джим Керри, 
    self.frame_list_file = os.path.join(DIR, 'result', 'test_frame_list.csv')  # Имя файла в котором сохраняются индексы кадров с Джимом Керри
    self.output_video_file = os.path.join(DIR, 'result', 'test_result.avi')
    self.output_video_file_mp4 = os.path.join(DIR, 'result', 'test_result.mp4')
    with open(self.frame_list_file, 'tw', encoding='utf-8') as f:  # Создаём пустой файл
      pass
    self.threshold = 0.6
    self.sp = dlib.shape_predictor(os.path.join(DIR, 'dlib_models', 'shape_predictor_68_face_landmarks.dat'))
    self.facerec = dlib.face_recognition_model_v1(os.path.join(DIR, 'dlib_models', 'dlib_face_recognition_resnet_model_v1.dat'))
    self.detector = dlib.get_frontal_face_detector()


  # Получаем тензор, содержащий только область лица. Возвращает список тензоров изображения и дескрипторов.
  def getRectangle(self, image_np):  # save - True / False - сохранять изображения и записывать дескрипторы
    # Находим изображение
    face_coord_list = self.detector(image_np, 1)  # Возвращает список координат найденных лиц
    # Перебираем все найденные лица на изображении
    img_rect_list = []
    face_descriptor_list = []
    for coord in face_coord_list:
      img_rect = image_np[coord.top():coord.bottom(), coord.left():coord.right()]  # Тензор лица
      img_rect_list.append(img_rect)
      shape = self.sp(image_np, coord)  # Получаем набор ключевых точек
      face_descriptor = self.facerec.compute_face_descriptor(image_np, shape)  # Извлекаем дескриптор из лица
      face_descriptor_list.append(face_descriptor)
    return img_rect_list, face_descriptor_list   # Возвращаем список изображений и дескрипторов


  # Рассчитываем Евклидово расстояние между векторами и возвращаем меру схожести
  def euclideanDistance(self, vector_1, vector_2):
    return distance.euclidean(vector_1, vector_2)


  def saveMovie(self, video_np):
    frame_list = []  # Список кадров с Джимом Керри
    for i, frame in enumerate(video_np):
      if i in self.frame_list:
        frame_list.append(frame)
    video_np = np.array(frame_list, dtype=np.float32) 
    print(video_np.shape)
    output = video_np.astype(np.uint8)
    skvideo.io.vwrite(self.output_video_file, output)
    skvideo.io.vwrite(self.output_video_file_mp4, output)


  # Отображение 3х изображений
  def showImage(self, image_list):
    fig, axs = plt.subplots(1, 3, figsize=(25, 10))
    axs[0].imshow(image_list[0])
    axs[1].imshow(image_list[1])
    axs[2].imshow(image_list[2])

In [6]:
DAN = DanFace()

# ОБУЧЕНИЕ


## Вектор лица для Джима Керри
Определим дескриптор. Данные получены в файле 'main.ipynb', здесь мы сразу их пропишем без вычисления.

In [7]:
DK = [-0.05413726717233658, 0.20258700847625732, -0.02980971336364746, -0.009266268461942673, -0.11678464710712433, 0.07381494343280792, -0.06925348937511444, -0.06044156849384308, 0.2005159556865692, 0.024151720106601715, 0.18836179375648499, 0.049733638763427734, -0.24746623635292053, 0.021895507350564003, -0.05165024474263191, 0.12912672758102417, -0.207529217004776, -0.0803472101688385, -0.1228303462266922, -0.04064009338617325, 0.049077291041612625, 0.0820658951997757, 0.044419772922992706, 0.08841884136199951, -0.13434597849845886, -0.30047282576560974, -0.06881625205278397, -0.17035093903541565, 0.01376345194876194, -0.2013019621372223, 0.004748144187033176, -0.030011259019374847, -0.20061665773391724, -0.10102177411317825, 0.011843681335449219, 0.07204631716012955, -0.16505970060825348, -0.1117887943983078, 0.2104894369840622, 0.10288297384977341, -0.10170906782150269, 0.03403836488723755, 0.017056070268154144, 0.33924412727355957, 0.18967221677303314, 0.007659764960408211, -0.005537357181310654, -0.07320354133844376, 0.10830501466989517, -0.25624531507492065, 0.11208046972751617, 0.1581788957118988, 0.1098909005522728, 0.18746814131736755, 0.08272372931241989, -0.15778550505638123, 0.07521936297416687, 0.14968138933181763, -0.18743184208869934, 0.04760957136750221, 0.06635124236345291, -0.012313809245824814, -0.013106746599078178, -0.030088935047388077, 0.1931372582912445, 0.12213562428951263, -0.027669906616210938, -0.14269167184829712, 0.07506205886602402, -0.07834555208683014, -0.1176898330450058, 0.10153452306985855, -0.13343235850334167, -0.19007247686386108, -0.2705255448818207, 0.040848083794116974, 0.3956400454044342, 0.1535823792219162, -0.1679103523492813, -0.060541026294231415, -0.11328165978193283, -0.04660944640636444, 0.03196923807263374, -0.06560832262039185, -0.08131857216358185, -0.06055719032883644, -0.1024099588394165, 0.09894369542598724, 0.2049059122800827, -0.02381899207830429, -0.08042197674512863, 0.19882942736148834, 0.025886990129947662, -0.03542086482048035, 0.06663700938224792, 0.028573982417583466, -0.08232629299163818, 0.018684521317481995, -0.16942471265792847, -0.07508149743080139, 0.0560985766351223, -0.08128245919942856, -0.0441010445356369, 0.0699792131781578, -0.2297079712152481, 0.13236193358898163, 0.0993364155292511, -0.0639580637216568, -0.009845227003097534, -0.01188822090625763, -0.2784838378429413, 0.03532838448882103, 0.286836177110672, -0.2830303907394409, 0.31698328256607056, 0.172643780708313, 0.05135684460401535, 0.08970710635185242, 0.060273509472608566, 0.025302646681666374, -0.08992411941289902, -0.07427418977022171, -0.16987451910972595, -0.06985627114772797, 0.11119832843542099, 0.07064186781644821, 0.031145533546805382, 0.013018637895584106]

## Загружаем тестовое видео

In [8]:
video_np = skvideo.io.vread(os.path.join(DIR, 'video', 'test_1.mp4'))  
print(video_np.shape)

(5391, 408, 720, 3)


## Поиск и запись кадров с Джимом Керри на всём видео

In [9]:
DAN.frame_list = []  # Очищаем список
time_start = time.time()
for i, image in enumerate(video_np):  # Перебираем кадры видео
  image_list, desc_list = DAN.getRectangle(video_np[i])

  for j, desc in enumerate(desc_list):  # Перебираем найденные лица на видео
    dist = DAN.euclideanDistance(desc, DK)
    if dist < DAN.threshold:  # Найден Джим Керри
      DAN.frame_list.append(i)
      continue

  if i % 100 == 0:
    delta_time = time.time() - time_start
    print(f'Найдено кадров с Джимом Керри: {len(DAN.frame_list)} из: {i}, время работы {round(delta_time)} c')

result_frame_list = DAN.frame_list
print('--- ИТОГО: ---')
print (f'Найдено кадров с Джимом Керри: {len(DAN.frame_list)} из: {i}')

Найдено кадров с Джимом Керри: 0 из: 0, время работы 0 c
Найдено кадров с Джимом Керри: 0 из: 100, время работы 25 c
Найдено кадров с Джимом Керри: 36 из: 200, время работы 44 c
Найдено кадров с Джимом Керри: 78 из: 300, время работы 63 c
Найдено кадров с Джимом Керри: 104 из: 400, время работы 82 c
Найдено кадров с Джимом Керри: 104 из: 500, время работы 101 c
Найдено кадров с Джимом Керри: 104 из: 600, время работы 120 c
Найдено кадров с Джимом Керри: 192 из: 700, время работы 139 c
Найдено кадров с Джимом Керри: 273 из: 800, время работы 157 c
Найдено кадров с Джимом Керри: 273 из: 900, время работы 176 c
Найдено кадров с Джимом Керри: 273 из: 1000, время работы 194 c
Найдено кадров с Джимом Керри: 274 из: 1100, время работы 212 c
Найдено кадров с Джимом Керри: 309 из: 1200, время работы 231 c
Найдено кадров с Джимом Керри: 314 из: 1300, время работы 251 c
Найдено кадров с Джимом Керри: 412 из: 1400, время работы 271 c
Найдено кадров с Джимом Керри: 476 из: 1500, время работы 290 c


## Выводим информацию

In [10]:
test_movie_len = i/25
test_movie_m = test_movie_len // 60
test_movie_s = test_movie_len % 60
print(f'Тестовый видеоролик длинной: {int(test_movie_m)}м, {round(test_movie_s)}c')

result_movie_len = len(DAN.frame_list)/25
result_movie_m = result_movie_len // 60
result_movie_s = result_movie_len % 60
print(f'Итоговый видеоролик длинной: {int(result_movie_m)}м, {round(result_movie_s)}c')

Тестовый видеоролик длинной: 3м, 36c
Итоговый видеоролик длинной: 0м, 47c


## Сохранение полученного видео с Джимом Керри

In [11]:
if len(DAN.frame_list) == 0:  # Если модель объект DAN инициализировали заново DAN.frame_list - обнуляется
  DAN.frame_list = result_frame_list

DAN.saveMovie(video_np)

(1164, 408, 720, 3)


# РОЛИК ГОТОВ.

Ролик находится тут 'files/single_result/test_result.avi' относительно папки проекта



In [13]:
from base64 import b64encode
mp4 = open(DAN.output_video_file_mp4, 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""<video width=720 controls><source src="%s" type="video/mp4"></video>""" % data_url)

Output hidden; open in https://colab.research.google.com to view.